The following code was tested only with Linux OS. The Oracle DB client was installed and the cx_Oracle python library was used with Python 3.5.

In [1]:
import cx_Oracle
import csv
import os

We set the language to UTF8 so we can import and read Unicode characters that exist in the dataset.

In [2]:
os.environ['NLS_LANG'] = '.AL32UTF8'
db = cx_Oracle.connect('user_1', 'babis', '52.50.54.24:1521/ORCL', encoding='utf-8')
print('Oracle DB server version %s' % db.version)

Oracle DB server version 12.1.0.2.0


Create the connection cursor we will use for all operations.

In [3]:
cursor = db.cursor()

We read the SQL script and process it command by command.

In [22]:
script_file = open('part2/Part 2.sql')
script = script_file.read()
commands = script.split(';')

for command in commands:
    if(command.isspace()==False and len(command) > 0):
        command = command.strip()
        cursor.execute(command)

To transpose the matrix we swap the rows with the columns

In [24]:
part2a = ('SELECT j AS i, i AS j, val '
          'FROM A '
          'ORDER BY i, j')

for i, row in enumerate(cursor.execute(part2a)):
    print(row)

(0, 0, 7)
(0, 1, 10)
(0, 2, 2)
(1, 0, 5)
(1, 1, 7)
(1, 2, 0)
(2, 0, 8)
(2, 1, 7)
(2, 2, 5)


To get the dot product we sum the product on corresponding elements:

In [30]:
part2b = ('SELECT SUM(A.val * B.val) '
          'FROM A JOIN B '
          'ON A.i=B.i '
          'WHERE A.j=1 AND B.j=2')

for i, row in enumerate(cursor.execute(part2b)):
    print(row)

(113,)


Same as above:

In [33]:
part2c = ('SELECT SUM(A.val * B.val) '
          'FROM A JOIN B '
          'ON A.j=B.i '
          'WHERE A.i=1 AND B.j=2'
         )

for i, row in enumerate(cursor.execute(part2c)):
    print(row)

(212,)


For matrix multiplication we get the dot product for each element in the matrix. We group by rows and columns to achieve that:

In [41]:
part2d = ('SELECT A.i AS i, B.j AS j, SUM(A.val * B.val) AS val '
          'FROM A '
          'JOIN B '
          'ON B.i=A.j '
          'GROUP BY A.i, B.j '
          'ORDER BY i, j'
         )

for i, row in enumerate(cursor.execute(part2d)):
    print(row)

(0, 0, 106)
(0, 1, 80)
(0, 2, 171)
(1, 0, 146)
(1, 1, 109)
(1, 2, 212)
(2, 0, 23)
(2, 1, 17)
(2, 2, 55)


We drop the tables and close the connection to the DB server.

In [21]:
drop_A = ('DROP TABLE A')
cursor.execute(drop_A)
drop_B = ('DROP TABLE B')
cursor.execute(drop_B)

In [42]:
cursor.close()
db.close()